### Import

In [1]:
import sqlite3
import requests
import re
from bs4 import BeautifulSoup, Comment
import json
import pandas as pd
import time
import folium
import matplotlib.pyplot as plt
import urllib.request
from selenium import webdriver
import random

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

### Web Scraping Beer Advocate

#### Selenium activation

In [ ]:
#testing that Selenium works
from selenium import webdriver
driver = webdriver.Chrome()
driver.get('https://www.nasa.gov')
headlines = driver.find_elements_by_class_name("headline")
for headline in headlines:
    print(headline.text.strip())
driver.close()

>Here I am doing a simple test run of the selenium driver to make sure it works buy pulling up a website, checking it, and then closing it.

In [3]:
#estabishing the webdriver for chrome
driver = webdriver.Chrome()

In [4]:
#putting the driver on the website
driver.get('https://www.beeradvocate.com/beer/styles/9/')

>The "/styles/9/" is the style of beer I will be looking at to start the project. It is "Fruit and Field Beer", but the number at the end of the styles can be changed such that it will represent the other styles, such as IPA or Lager.

In [5]:
#clicking into a beer
driver.find_element_by_xpath('//*[@id="ba-content"]/table/tbody/tr[4]/td[1]/a').click()

>This is code that allows us to click on the first element of the table and look at it.

In [8]:
#pulling out the comments
comment = driver.find_element_by_xpath('//*[@id="rating_fullview_content_2"]').text
comment

"Elemental19xx from Illinois\n\n3.6/5  rDev +5.3%\nlook: 3 | smell: 3.5 | taste: 3.75 | feel: 3.75 | overall: 3.5\n\nBeer pours a little cloudy with a little head, and a few small particulates. Smells more like a fruity ale than an IPA (which I like). tastes pretty good in my opinion, but not as hoppy as a pale ale( perhaps that's why it says not quite pale ale on the front). It's like your average run of the mill ale, but just a bit more fruity and a light hop flavor towards the end. The carbonation is mild, not smooth like nitro brews, but relatively soft. Overall I could drink em all day and not be sick of the taste, but it would be my first choice if I was looking to only drink a beer or two after dinner. That being said if I see them in the custom six pack fridge I would be okay with adding one of these.\n\nFeb 28, 2021"

>This will allow us to grab the first comment after clicking into the beer profile

In [10]:
#this function will let us go from page to page
def next_page():
    quarter = driver.find_element_by_tag_name('tbody')
    x = quarter.find_elements_by_tag_name('span')[1].find_elements_by_tag_name('a')[-2]
    x.click()

>When on the style page this will allow us to move from page to page.

In [ ]:
driver.close()

#### BeautifulSoup activation

In [11]:
#establish our url and our request.get
url = 'https://www.beeradvocate.com/beer/styles/9/'
response = requests.get(url)

In [12]:
#establishing beautifulsoup with the response variable using 'lxml' and then grabbing the table
bs = BeautifulSoup(response.content, 'lxml')
table = bs.table

In [13]:
#using the table we established to grab all the specific values out of that table
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

['Style Examples - 1 to 50 (out of 7214) - Ordered by # Reviews']
['\nfirst ← prev |\xa01-50\xa0|\xa051-100\xa0|\xa0101-150\xa0 | next → last']
['\nName\n', '\nBrewery\n', '\nABV\n', '\nRatings\xa0>\n', '\nAvg\n']
['#9', 'Magic Hat Brewing Company', '5.10', '5,915', '3.42', '']
['Samuel Adams Cherry Wheat', 'Boston Beer Company (Samuel Adams)', '5.30', '4,029', '3.19', '']
['Raspberry Tart', 'New Glarus Brewing Company', '4.00', '3,995', '4.41', '']
['Summer Shandy', 'Jacob Leinenkugel Brewing Company', '4.20', '3,841', '3.2', '']
['Rübæus', 'Founders Brewing Company', '5.70', '3,800', '3.91', '']
['Wisconsin Belgian Red', 'New Glarus Brewing Company', '4.00', '3,782', '4.42', '']
['Purple Haze', 'Abita Brewing Co.', '4.20', '3,529', '3.32', '']
['Banana Bread Beer', 'Eagle Brewery', '5.20', '3,235', '3.56', '']
['Serendipity', 'New Glarus Brewing Company', '4.00', '2,994', '4.38', '']
['Hell Or High Watermelon Wheat Beer', '21st Amendment Brewery', '4.90', '2,948', '3.4', '']
['Blushi

>This will output the information of the table on the page

In [14]:
#here we put it into a df
df = pd.read_html(str(table))
df = df[0].dropna(axis= 0, thresh = 4)
df

,0,1,2,3,4,5
0,Style Examples - 1 to 50 (out of 7214) - Order...,Style Examples - 1 to 50 (out of 7214) - Order...,Style Examples - 1 to 50 (out of 7214) - Order...,Style Examples - 1 to 50 (out of 7214) - Order...,Style Examples - 1 to 50 (out of 7214) - Order...,NaN
1,first ← prev | 1-50 | 51-100 | 101-150 | next ...,first ← prev | 1-50 | 51-100 | 101-150 | next ...,first ← prev | 1-50 | 51-100 | 101-150 | next ...,first ← prev | 1-50 | 51-100 | 101-150 | next ...,first ← prev | 1-50 | 51-100 | 101-150 | next ...,NaN
2,Name,Brewery,ABV,Ratings >,Avg,NaN
3,#9,Magic Hat Brewing Company,5.10,5915,3.42,NaN
4,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,4029,3.19,NaN
5,Raspberry Tart,New Glarus Brewing Company,4.00,3995,4.41,NaN
6,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,3841,3.2,NaN
7,Rübæus,Founders Brewing Company,5.70,3800,3.91,NaN
8,Wisconsin Belgian Red,New Glarus Brewing Company,4.00,3782,4.42,NaN
9,Purple Haze,Abita Brewing Co.,4.20,3529,3.32,NaN


>Taking the data from above and creating a dataframe to look at it

In [ ]:
#This allows us to create an accurate range for the function to iterate through
body = driver.find_element_by_tag_name('tbody')
x = body.find_element_by_tag_name('b').text
pages = int(''.join([i for i in x.split('(')[1] if i.isnumeric()])) // 50

>We need to be able to accurately pull the range for any of the styles of the beer we look at. This will give an exact range for the number of pages the function needs to go through when pulling the data from the style pages.

#### Establish table with beer info

In [ ]:
#creating a variable to put into the next function
long_list = [('Fruit and Field', 'https://www.beeradvocate.com/beer/styles/9/', pages)]

>This variable is setup to eventually be more variable to put in more than just the "Fruit and Field" beers and that url. Similarly the url can be adjusted to iterate through all the styles we care about.

In [ ]:
#this function will go through each page and generate the tables iteratively
#it has a sleep timer to avoid being blocked
data = []
for beer_style, url, pages in long_list:
    driver.get(url)
    for i in tqdm(range(pages)):
        print(i)
        table = driver.find_element_by_tag_name('table')
        tds = [x.find_elements_by_tag_name('td') for x in table.find_elements_by_tag_name('tr')][3:]
        for td in tds:
            data.append([beer_style] + [x.text for x in td] + [td[0].find_element_by_tag_name('a').get_attribute('href')])
            
        quarter = driver.find_element_by_tag_name('tbody')
        x = quarter.find_elements_by_tag_name('span')[1].find_elements_by_tag_name('a')[-2]
        x.click()
        time.sleep(random.choice([x/10 for x in range(8,14)]))

>In this function we are pooling a lot of the previous elements together to a cohesive dataset. 

>First we are grabbing the 'table' that the data is in and then iteratively pulling the data from those rows and then in the next loop filling out the data list with the specific elements on the page. 

>Secondly we have a loop that will move to the next page after the former is done.

>Finally we have a random sleep timer to prevent the website from blocking this function.

In [ ]:
#turning the data into a dataframe
beer_df = pd.DataFrame(data)

In [ ]:
#creating a benchmark csv in casae something happens
beer_df.to_csv(r'C:\Users\GMoneyMan\Documents\Flatiron\capstone\csv_data\beer_df.csv', index = False)

>Now that we have a simple Dataframe we save it just in case, since the website is finicky 

In [ ]:
beer_df.isna().sum()

In [ ]:
len(beer_df)

#### Collecting url for each beer profile

In [ ]:
#this function will create a list of each of the beer's url and using BS go to the next page
new_list_links = []

for tag in beer_tags.find_all(href=re.compile("/beer/styles/[0-9]+/\?sort=revsD")):
        if tag.contents[0] == 'last':
            num = int(tag.get('href').strip(f"{url}?sort=revsD&start"))
some_num = 7206 #num+50
i = 0
for i in tqdm(range(0, some_num, 50)):
        print(f'i is {i}')
        url = f'https://www.beeradvocate.com/beer/styles/9/?sort=revsD&start={i}'
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        tags = [tag.get('href') for tag in soup.find_all(href=re.compile("/beer/profile/[0-9]+/[0-9]+"))]
        new_list_links.append(tags)
        i += 50
        
        time.sleep(random.choice([x/10 for x in range(8,14)]))

>Now We want to grab the reviews for each of the beers. In order to do that we need to get the url endings for each of the beers so we can create a function that can go into to those urls and grab the reviews.

>This does that by going into the rows and pulling the url endings, and then using BeautifulSoup going to the next page since it goes up by 50 beers a page. 

In [ ]:
new_list_links

In [ ]:
#Since the function outputs a list full of lists this will turn everything into just one list
list_of_beer_profiles = [y for x in new_list_links for y in x]

#### Collecting the reviews for a beer

>Each of the following methods extract the reviews from the pages, but some do so with no lists, one list, or multiple lists. I go with the single list for easier access and application.

##### Method 1 to get reviews

In [ ]:
tag_name = driver.find_element_by_xpath('//*[@id="ba-content"]/table/tbody/tr[4]/td[1]/a').text
tag_name

##### Method 2 to get reviews

In [ ]:
ba_content = driver.find_element_by_xpath('//*[@id="ba-content"]').text
ba_content

##### Method 3 to get reviews

In [ ]:
u_comment = driver.find_elements_by_class_name('user-comment')
review3 = []
for item in u_comment:
    review3.append(item.text)

#### Collecting the url & reviews

In [ ]:
#creating a test for the urls to feed into the next function
first_trial = list_links[0][49:52]

>Since the review scraping is very intensive and takes a long time, I create a smaller sample of the urls we will need to test that it does what it is supposed to.

In [ ]:
#this function will take the list of urls collected previously and pull all the comments from
#the page, and append the text to a list
def get_beer_reviews(list_urls):
    list_beers = []
    i = 1
    for url_end in tqdm(list_urls):
        beer = {}
        url = f'https://www.beeradvocate.com{url_end}'
        beer['url'] = url
        print(i)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser') 
        
        listy = soup.find_all(class_ = 'user-comment')
        beer['review'] = []
        for item in listy:
            beer['review'].append(item.get_text())
            
#         titlebar = soup.find(class_='titleBar')
#         for span in titlebar('span'):
#             span.decompose()

#         beer['name'] = titlebar.get_text().strip()
        
        list_beers.append(beer)
        
        time.sleep(random.choice([x/10 for x in range(8,14)]))
        
        i += 1
    return list_beers

>This function will go through all the of the urls in the list I pulled for each of the beers. It will go into that url and pull all of the reviews on the page and append them to a new list. This also had a random sleep timer as this whole function took a long time. 

In [ ]:
#here we trial our sample of urls
demo_df = get_beer_reviews(first_trial)

>Here we try a small sample of the urls in the list to make sure it works.

In [ ]:
pd.DataFrame(demo_df)

In [ ]:
len(list_of_beer_profiles)

In [ ]:
#here we use our whole list of urls (this takes a long time)
reviews = get_beer_reviews(list_of_beer_profiles)

>Since the sample worked I can now apply the function for the entire list of urls

In [ ]:
#we convert into a dataframe
beer_reviews = pd.DataFrame(reviews)
beer_reviews

In [ ]:
beer_reviews['review'][7201]

In [ ]:
#this converts all the empty lists in our review column into nan values
beer_reviews.loc[~beer_reviews.review.astype(bool),'review']=np.nan

>Here we take any of the empty lists, generated for when there are no reviews, and turn them into NaN values to get rid of easily.

In [ ]:
beer_reviews.isna().sum()

In [ ]:
beer_reviews.to_csv(r'C:\Users\GMoneyMan\Documents\Flatiron\capstone\csv_data\reviews.csv', index = False)

In [ ]:
#here we merge the reviews to the beer dataframe on the url column
df = beer_df.merge(beer_reviews, on='url', how='left')
df.to_csv(r'C:\Users\GMoneyMan\Documents\Flatiron\capstone\csv_data\beer_w_reviews.csv', index = False)

>Now we merge on the url for the beer, as they are the same.

In [ ]:
df

In [ ]:
#we renaming the columns
df.columns = ['beer_style', 'beer_name', 'brewery_name', 'abv', 'no_ratings', 'avg_rating', 'drop_me', 'url']